In [19]:
import pandas as pd
import requests
import time
import os
from dotenv import load_dotenv

In [ ]:
# Authentication header required for requests to the football-data.org API
# Loading API key from environment variables

load_dotenv()
API_KEY = os.getenv("API_KEY")

headers = { "X-Auth-Token": API_KEY }

In [3]:
# See all available competitions (leagues) data  
url_competitions = "https://api.football-data.org/v4/competitions/"
r_competitions = requests.get(url_competitions, headers=headers)
data_competitions = r_competitions.json()

# data["competitions"] contains all leagues
competitions = data_competitions["competitions"]
df_competitions = pd.json_normalize(competitions)
leagues = df_competitions[["id", "name", "code", "type"]]
leagues

# We will collect match data for the following leagues
# Premier League (England)
# La Liga (Spain)
# Serie A (Italy)
# Bundesliga (Germany)
# Ligue 1 (France)

,id,name,code,type
0,2013,Campeonato Brasileiro Série A,BSA,LEAGUE
1,2016,Championship,ELC,LEAGUE
2,2021,Premier League,PL,LEAGUE
3,2001,UEFA Champions League,CL,CUP
4,2018,European Championship,EC,CUP
5,2015,Ligue 1,FL1,LEAGUE
6,2002,Bundesliga,BL1,LEAGUE
7,2019,Serie A,SA,LEAGUE
8,2003,Eredivisie,DED,LEAGUE
9,2017,Primeira Liga,PPL,LEAGUE


## Fetching Data

In [4]:
years = [2024,2023] # Last two seasons
league_ids = [2021,2019,2002,2015,2014] # League IDs

In [5]:
all_matches = [] # List to hold all match data

# This function retrieves all teams in a league for a specific season
# Then, for each team, it retrieves all matches played in that season
def get_teams_and_matches(league_id, year):
    
    # Fetch team names and id's
    url_teams = f"https://api.football-data.org/v4/competitions/{league_id}/teams?season={year}"
    response_teams = requests.get(url_teams, headers=headers)
    data_teams = response_teams.json()
    
    season_teams = data_teams["teams"]
    df_teams = pd.json_normalize(season_teams)

    # Retrieves all matches played in season and put into a list
    for team_id, team_shortname in zip(df_teams["id"], df_teams["shortName"]):
        url_matches = f"https://api.football-data.org/v4/teams/{team_id}/matches?season={year}&competitions={league_id}"
        response_matches = requests.get(url_matches, headers=headers)
        data_matches = response_matches.json()

        # Check if matches data is available 
        if "matches" in data_matches:
            df_team_matches = pd.json_normalize(data_matches["matches"])
            
            # Organize the data for the predictor
            df_team_matches["team"] = team_shortname
            df_team_matches["opponent"] = df_team_matches.apply(lambda row: row["awayTeam.shortName"] if row["homeTeam.shortName"] == team_shortname else row["homeTeam.shortName"], axis=1)
            df_team_matches["venue"] = df_team_matches.apply(lambda row: "Home" if row["homeTeam.shortName"] == team_shortname else "Away", axis=1)
            df_team_matches["result"] = df_team_matches.apply(lambda row: "W" if (row["score.winner"] == "HOME_TEAM" and row["venue"] == "Home") or
                                         (row["score.winner"] == "AWAY_TEAM" and row["venue"] == "Away") else
                                         ("D" if row["score.winner"] == "DRAW" else "L"), axis=1)
            df_team_matches[["gf", "ga"]] = df_team_matches.apply(lambda row: pd.Series([row["score.fullTime.home"], row["score.fullTime.away"]]) if row["venue"] == "Home"
                                   else pd.Series([row["score.fullTime.away"], row["score.fullTime.home"]]), axis=1)
            df_team_matches["goal_diff"] = df_team_matches["gf"] - df_team_matches["ga"]

            all_matches.append(df_team_matches)

        time.sleep(8) # avoid hitting API rate limits

In [6]:
# Loop through each league and year to collect match data
for league in league_ids:
    for year in years:
        get_teams_and_matches(league, year)

# Concatenate all match data into a single DataFrame
df_all_matches = pd.concat(all_matches, ignore_index=True)

## Data cleaning and transformation

In [7]:
df_all_matches.columns

Index(['id', 'utcDate', 'status', 'matchday', 'stage', 'group', 'lastUpdated',
       'referees', 'area.id', 'area.name', 'area.code', 'area.flag',
       'competition.id', 'competition.name', 'competition.code',
       'competition.type', 'competition.emblem', 'season.id',
       'season.startDate', 'season.endDate', 'season.currentMatchday',
       'season.winner', 'homeTeam.id', 'homeTeam.name', 'homeTeam.shortName',
       'homeTeam.tla', 'homeTeam.crest', 'awayTeam.id', 'awayTeam.name',
       'awayTeam.shortName', 'awayTeam.tla', 'awayTeam.crest', 'score.winner',
       'score.duration', 'score.fullTime.home', 'score.fullTime.away',
       'score.halfTime.home', 'score.halfTime.away', 'odds.msg', 'team',
       'opponent', 'venue', 'result', 'gf', 'ga', 'goal_diff',
       'season.winner.id', 'season.winner.name', 'season.winner.shortName',
       'season.winner.tla', 'season.winner.crest', 'season.winner.address',
       'season.winner.website', 'season.winner.founded',
       '

In [8]:
# Select relevant columns for analysis
df = df_all_matches[[
        #'id', 
        'utcDate', 
        #'status', 
        'matchday', 
        #'stage', 'group', 'lastUpdated', 'referees', 'area.id', 
        'area.name', 
        #'area.code', 'area.flag', 'competition.id',
        'competition.name', 
        #'competition.code',
        #'competition.type', 'competition.emblem', 'season.id',
        #'season.startDate', 'season.endDate', 'season.currentMatchday', 'season.winner', 
        #'homeTeam.id', 'homeTeam.name', 'homeTeam.shortName',
        #'homeTeam.tla', 'homeTeam.crest', 'awayTeam.id', 'awayTeam.name',
        #'awayTeam.shortName', 'awayTeam.tla', 'awayTeam.crest', 
        #'score.winner', 'score.duration', 'score.fullTime.home', 'score.fullTime.away',
        #'score.halfTime.home', 'score.halfTime.away', 'odds.msg', 
        'venue', 'team', 'gf', 'ga', 'opponent', 'result', 'goal_diff',
        #'season.winner.id', 'season.winner.name', 'season.winner.shortName',
        #'season.winner.tla', 'season.winner.crest', 'season.winner.address',
        #'season.winner.website', 'season.winner.founded',
        #'season.winner.clubColors', 'season.winner.venue',
        #'season.winner.lastUpdated'
        ]]

In [9]:
# Copy the dataframe to avoid warnings
df = df.copy()
df.head(20)

,utcDate,matchday,area.name,competition.name,venue,team,gf,ga,opponent,result,goal_diff
0,2024-08-17T14:00:00Z,1,England,Premier League,Home,Arsenal,2,0,Wolverhampton,W,2
1,2024-08-24T16:30:00Z,2,England,Premier League,Away,Arsenal,2,0,Aston Villa,W,2
2,2024-08-31T11:30:00Z,3,England,Premier League,Home,Arsenal,1,1,Brighton Hove,D,0
3,2024-09-15T13:00:00Z,4,England,Premier League,Away,Arsenal,1,0,Tottenham,W,1
4,2024-09-22T15:30:00Z,5,England,Premier League,Away,Arsenal,2,2,Man City,D,0
5,2024-09-28T14:00:00Z,6,England,Premier League,Home,Arsenal,4,2,Leicester City,W,2
6,2024-10-05T14:00:00Z,7,England,Premier League,Home,Arsenal,3,1,Southampton,W,2
7,2024-10-19T16:30:00Z,8,England,Premier League,Away,Arsenal,0,2,Bournemouth,L,-2
8,2024-10-27T16:30:00Z,9,England,Premier League,Home,Arsenal,2,2,Liverpool,D,0
9,2024-11-02T12:30:00Z,10,England,Premier League,Away,Arsenal,0,1,Newcastle,L,-1


In [10]:
# Rename columns for better readability
df = df.rename(columns={"utcDate":"date", "matchday":"week", "area.name": "country", "competition.name":"league"})

In [11]:
# Standardize league and team names
df["league"] = df["league"].replace("Primera Division", "LaLiga")
df["team"] = df["team"].replace("Barça", "Barcelona")
df["opponent"] = df["opponent"].replace("Barça", "Barcelona")

In [13]:
# Convert date column to datetime and create separate columns for date and hour
df["date"] = pd.to_datetime(df["date"])
df["match_date"] = df["date"].dt.date
df["match_hour"] = df["date"].dt.hour  # this leaves only hours
df["day"] = df["date"].dt.day_name()

df = df.drop(columns=["date"])

In [14]:
# Create a season column
df["season"] = df["match_date"].apply(lambda date: f"{date.year}/{date.year + 1}" if date.month >= 8 else f"{date.year - 1}/{date.year}")

In [15]:
# Reorder the columns
cols = df.columns.tolist()
cols.remove("match_date")
cols.remove("match_hour")
cols.remove("season")
cols.remove("day")
new_order = ["match_date", "day", "match_hour"] + cols[:2] + ["season"] + cols[2:]

# Reorder the dataframe
df = df[new_order]

In [20]:
df.head(10)

,match_date,day,match_hour,week,country,season,league,venue,team,gf,ga,opponent,result,goal_diff
0,2024-08-17,Saturday,14,1,England,2024/2025,Premier League,Home,Arsenal,2,0,Wolverhampton,W,2
1,2024-08-24,Saturday,16,2,England,2024/2025,Premier League,Away,Arsenal,2,0,Aston Villa,W,2
2,2024-08-31,Saturday,11,3,England,2024/2025,Premier League,Home,Arsenal,1,1,Brighton Hove,D,0
3,2024-09-15,Sunday,13,4,England,2024/2025,Premier League,Away,Arsenal,1,0,Tottenham,W,1
4,2024-09-22,Sunday,15,5,England,2024/2025,Premier League,Away,Arsenal,2,2,Man City,D,0
5,2024-09-28,Saturday,14,6,England,2024/2025,Premier League,Home,Arsenal,4,2,Leicester City,W,2
6,2024-10-05,Saturday,14,7,England,2024/2025,Premier League,Home,Arsenal,3,1,Southampton,W,2
7,2024-10-19,Saturday,16,8,England,2024/2025,Premier League,Away,Arsenal,0,2,Bournemouth,L,-2
8,2024-10-27,Sunday,16,9,England,2024/2025,Premier League,Home,Arsenal,2,2,Liverpool,D,0
9,2024-11-02,Saturday,12,10,England,2024/2025,Premier League,Away,Arsenal,0,1,Newcastle,L,-1


In [21]:
df.shape

(6098, 14)

In [22]:
# Save the DataFrame as CSV for the predictor
df.to_csv("top_5_leagues_match_data.csv", index=False)